In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
# use process bar tool
from tqdm import tqdm
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-final-dataset/dataset/test/adults/20.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/6.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/76.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/71.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/5.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/8.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/84.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/85.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/67.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/82.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/30.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/10.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/0.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/62.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/61.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/73.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/60.jpg
/kaggle/input/ml-f

In [2]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/test', transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [30]:
class ModifiedInceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(ModifiedInceptionModule, self).__init__()

        self.conv_1x1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.depthwise_3x3 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)
        self.pointwise_3x3 = nn.Conv2d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        path1 = self.conv_1x1(x)
        path2 = self.depthwise_3x3(x)
        path2 = self.pointwise_3x3(path2)
        outputs = torch.cat([path1, path2], dim=1)
        outputs = outputs[:, :x.size(1), :, :]  # Keep the same number of channels as the input
        outputs += x
        return F.relu(outputs)

class DeepNetwork(nn.Module):
    def __init__(self, num_classes=2, num_inception=5):
        super(DeepNetwork, self).__init__()

        self.initial_conv = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.downsample = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)

        #self.inception1 = ModifiedInceptionModule(32)
        #self.inception2 = ModifiedInceptionModule(32)
        #self.inception3 = ModifiedInceptionModule(32)
        #self.inception4 = ModifiedInceptionModule(32)
        self.inception = nn.ModuleList([ModifiedInceptionModule(64) for _ in range(num_inception)])
        for i in range(num_inception):
            self.inception.append(ModifiedInceptionModule(64))

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = F.relu(self.downsample(x))
        
        #x = self.inception1(x)
        #x = self.inception2(x)
        #x = self.inception3(x)
        #x = self.inception4(x)
        for i in range(len(self.inception)):
            x = self.inception[i](x)
            
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [36]:
# Model, loss function, optimizer
model = DeepNetwork(num_classes=2, num_inception = 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Transfer model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch [1/10], Loss: 0.7533
Epoch [2/10], Loss: 0.6900
Epoch [3/10], Loss: 0.6802
Epoch [4/10], Loss: 0.6718
Epoch [5/10], Loss: 0.6694
Epoch [6/10], Loss: 0.6599
Epoch [7/10], Loss: 0.6662
Epoch [8/10], Loss: 0.6640
Epoch [9/10], Loss: 0.6535
Epoch [10/10], Loss: 0.6588


In [37]:
# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 59.17%


In [38]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Train Accuracy: {100 * correct / total:.2f}%')

Train Accuracy: 59.64%


In [16]:
%pip install torchsummary torchprofile

Note: you may need to restart the kernel to use updated packages.


In [33]:
from torchsummary import summary
import torchvision.models as models
from torchprofile import profile_macs
# Calculate the number of trainable parameters and all parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print('trainable_params = ', trainable_params)
print('all_params = ', all_params)

# Create a dummy input tensor with the same size as the input images
input_tensor = torch.randn(1, 3, 224, 224).to(device)

# Calculate FLOPs
flops = profile_macs(model, input_tensor)
print(f'FLOPs: {flops / 1e9:.2f} GFLOPs')  # Convert to GFLOPs (GigaFLOPs)

trainable_params =  218050
all_params =  218050
FLOPs: 0.69 GFLOPs


In [ ]:
torch.save(model, '/kaggle/working/CustomModel0616.pt')